In [1]:
import pandas as pd
from geopy.geocoders import GoogleV3
from math import radians, sin, cos, sqrt, atan2
import os

In [2]:
os.path.abspath('../../') + '/data/geocode_tama_address_data.csv'

'/Users/sararozin/Documents/Tama/Tama/total_approved_request_per_year/data/geocode_tama_address_data.csv'

In [3]:
tama_df = pd.read_csv('/Users/sararozin/Documents/Tama/Tama/total_approved_request_per_year/data/geocode_tama_address_data.csv')
tama_df.columns

Index(['Unnamed: 0', 'MezheYeshut', 'GISId', 'MisparYeshut', 'MisparYeshutAv',
       'AchuzHachtamot', 'AchuzHachtamotMitcham', 'Bayt', 'GushChelka',
       'KamutDirot', 'KamutTziburi', 'kaymetminhelet', 'KodMinhelet',
       'KtovetAishKesher', 'MachozMinhal', 'MahutBakasha', 'MechirMemutza',
       'mitve', 'Rechov', 'SemelIishuv', 'ShemMitcham', 'ShnatBakasha',
       'ShnatHryter', 'ShnatIchlus', 'Status', 'TaarichMechiraRishona',
       'YachadKayam', 'YachadMutza', 'YachadTabo', 'YachadTosafti', 'Yazam',
       'Yeshut', 'Yeshuv', 'BakashaLeHeter', 'Heter1', 'Ichlus1', 'address',
       'coordinates', 'latitude', 'longitude'],
      dtype='object')

In [4]:
def get_city_coordinates(city):
    from geopy.geocoders import Nominatim
    # Create a geocoder instance
    geolocator = Nominatim(user_agent="my_app")
    # Geocode the city to get the location information
    location = geolocator.geocode(city, language="he")
    # Extract the latitude and longitude from the location data
    latitude = location.latitude
    longitude = location.longitude
    return longitude, latitude


def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers

    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Calculate the differences in latitude and longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Apply the Haversine formula
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance


def get_nearest_city(longitude, latitude):
    
    # Calculate the distance from the reference point for each row in the dataframe
    tama_df['distance'] = tama_df.apply(lambda row: calculate_distance(latitude, longitude, row['latitude'], row['longitude']), axis=1)

    # Find the row with the minimum distance
    nearest_point = tama_df.loc[tama_df['distance'].idxmin()]

    # Print the nearest point
    nearest_city = nearest_point['Yeshuv']
    print('nearest city', nearest_city)
    return nearest_city
    

def zoom_out(city):
    if city not in list(set(tama_df['Yeshuv'])):
        print('zoom out')
        longitude, latitude = get_city_coordinates(city)
        return get_nearest_city(longitude, latitude)
    return city

In [5]:
zoom_out('בני ברק')

'בני ברק'

In [6]:
tama_df = tama_df[tama_df['Yeshuv'] != 'בני ברק']

In [7]:
zoom_out('בני ברק')

zoom out
nearest city רמת גן


'רמת גן'

In [8]:
def get_city_code(city):
    city_map_name_to_code = tama_df[['Yeshuv', 'SemelIishuv']].drop_duplicates()
    city_map_name_to_code = city_map_name_to_code[city_map_name_to_code['Yeshuv'] == city]['SemelIishuv']
    return int(city_map_name_to_code)

In [9]:
get_city_code('רמת גן')

8600

In [10]:
def get_city_name(city):
    city_map_name_to_code = tama_df[['Yeshuv', 'SemelIishuv']].drop_duplicates()
    city_map_name_to_code = city_map_name_to_code.loc[city_map_name_to_code['SemelIishuv'] == city, 'Yeshuv']
    return list(city_map_name_to_code.values)[0]

In [11]:
get_city_name(8600)

'רמת גן'